In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from lnn import LNN, Formula, Or
import torch

model = LNN(alpha=0.5, learning=True) # minimum global alpha
n, f = 10, 3
f = torch.randint(1, n, (f,)) # randomise false indices
props = torch.stack([Formula.false()]*n) # gather true arguments
props[f] = Formula.true() # set false arguments

# loading facts and rules
for i in range(len(props)):
    model[str(i)][()] = props[i] # initialise bound truths
prop_str = list(map(str, list(range(len(props))))) # arguments as list of strings
model['Or'] = Or(*prop_str, lnn=model) # unpack proposition arguments 
model['Or'][()] = Formula.false()  # initialise the disjunction proposition and set rule to True

print_graph = lambda model: model.downward(func='print_graph', weights=True) # print the entire graph + neuron weights

# coefficients flags [unsupervised (contradictions), supervised (targets)]
model.train(coefficients=torch.tensor([1., 0.])) 

# uncomment below to see graph updates at print_epoch, as the network trains
# model.train(print_epoch=1, verbose=True, prediction_fn=print_graph, coefficients=torch.tensor([1., 0.])) 

print_graph(model)

# evaluate model
success = True
if str(model['Or'].state()) != 'False':
    for i in range(len(props)):
        if model[str(i)].state() != str(bool((model['Or'].weights[i]).round())):
            success = False
            print(i, lnn[str(i)].state(), '!=', str(bool((lnn['Or'].weights[i]).round())))
if success: print('\nsuccess')


converged at epoch 4
loss: 3.0 
loss_terms: tensor([3., 0.]) 
inference converged: True 
weights converged: True 
bounds tightened: tensor([0.0110, 0.0000])
Or
() False tensor([0.0903, 0.0000], grad_fn=<StackBackward>) 
weights: tensor([1., 1., 1., 0., 1., 0., 1., 1., 1., 0.], requires_grad=True)
9
() Contradiction tensor([1., 0.], grad_fn=<StackBackward>) 
8
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
7
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
6
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
5
() Contradiction tensor([1., 0.], grad_fn=<StackBackward>) 
4
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
3
() Contradiction tensor([1., 0.], grad_fn=<StackBackward>) 
2
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
1
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 
0
() False tensor([0.0909, 0.0000], grad_fn=<StackBackward>) 

success
